In [ ]:
import sys

import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import torch.nn.functional as F

import numpy as np
import glob
from tqdm import tqdm
import umap.umap_ as umap
import matplotlib.pyplot as plt

sys.path.append('../')

import loader.load_from_h5 as loadh5
from model.model import MyModel as model

In [ ]:
data_file = glob.glob('../data/normal*.h5')
train_loader = loadh5.FeatureDataset(data_file).get_dataloader(split_type='training')
val_loader = loadh5.FeatureDataset(data_file).get_dataloader(split_type='validation')
test_loader = loadh5.FeatureDataset(data_file).get_dataloader(split_type='test')

In [ ]:
model = model().to('cuda')

In [ ]:
c_criterion = nn.NLLLoss()
t_criterion = nn.TripletMarginLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)

In [ ]:
epochs = 20

In [ ]:
model.eval()
tst_embedded = []
tst_y = []
for x, y in test_loader:
    x = Variable(x.float()).to('cuda')
    embedded = model.embedding(x.view(-1,784)).detach().cpu()
    tst_embedded.append(embedded)
    tst_y.append(y)
tst_embedded = torch.cat(tst_embedded)
tst_y = torch.cat(tst_y)
#print(tst_embedded.shape,tst_y.shape)
hle = umap.UMAP(random_state=0,metric='euclidean',n_components=2,n_neighbors=20,min_dist=0).fit_transform(tst_embedded)
c_lst = [plt.cm.nipy_spectral(a) for a in np.linspace(0.0, 1.0, len(np.unique(tst_y)))]
plt.figure(figsize=(10,10))
for i in range(0,len(np.unique(tst_y))):
    #print('Class ',i,'x :',hle[tst_y==i,0])
    #print('Class ',i,'y :',hle[tst_y==i,1])
    plt.scatter(hle[tst_y==i,0],hle[tst_y==i,1],label=i,color=c_lst[i])
plt.legend(loc='best')
plt.title('UMAP 2D Before Training')
#plt.savefig('../result/model_ownAnomaly_beforeTraining.png')
plt.show()

In [ ]:
# model.eval()
# with torch.no_grad():
#     for x ,y in tqdm(train_loader):
#         triplet_tensor = []
#         triplet_label = []
#         for y_uni in torch.unique(y):
#             #print(y_uni)
#             anchor_x, anchor_y = x[y==y_uni], y[y==y_uni]
#             positive_x, positive_y = x[y==y_uni], y[y==y_uni]
#             negative_x, negative_y = x[y!=y_uni], y[y!=y_uni]
#             #print(anchor.shape,positive.shape,negative.shape)
#             a_ind = torch.arange(0,len(anchor_x))
#             p_ind = torch.arange(0,len(positive_x))
#             n_ind = torch.arange(0,len(negative_x))
#             triplet_ind = torch.cartesian_prod(a_ind,p_ind,n_ind)
#             #print(triplet_ind.shape)
        
#             for a,p,n in triplet_ind:
#                 if a != p:
#                     #print(anchor_x[a].shape,positive_x[p].shape,negative_x[n].shape)
#                     triplet_tensor.append(torch.stack([anchor_x[a],positive_x[p],negative_x[n]]))
#                     triplet_label.append(torch.stack([anchor_y[a],positive_y[p],negative_y[n]]))
#                     #break
#         triplet_tensor = torch.stack(triplet_tensor)
#         triplet_label = torch.stack(triplet_label)
        
#         anchor = model(Variable(triplet_tensor[:,0].float()).to('cuda'))
#         positive = model(Variable(triplet_tensor[:,1].float()).to('cuda'))
#         negative = model(Variable(triplet_tensor[:,2].float()).to('cuda'))
        
#         #print(t_criterion(anchor,positive,negative))

In [ ]:
train_loss=[]
val_loss = []
for epoch in tqdm(range(1,epochs+1)):
#for epoch in range(1,epochs+1):    
    model.train()
    b_loss=[]
    for x ,y in train_loader:
        x = Variable(x.float()).to('cuda')
        y = Variable(y.type(torch.LongTensor)).to('cuda')
        
        out = model(x)
        #print(x.shape,out.shape)
        triplet_tensor = []
        triplet_label = []
        for y_uni in torch.unique(y):
            #print(y_uni)
            anchor_x, anchor_y = out[y==y_uni], y[y==y_uni]
            positive_x, positive_y = out[y==y_uni], y[y==y_uni]
            negative_x, negative_y = out[y!=y_uni], y[y!=y_uni]
            #print(anchor.shape,positive.shape,negative.shape)
            a_ind = torch.arange(0,len(anchor_x))
            p_ind = torch.arange(0,len(positive_x))
            n_ind = torch.arange(0,len(negative_x))
            triplet_ind = torch.cartesian_prod(a_ind,p_ind,n_ind)
            #print(triplet_ind.shape)
        
            for a,p,n in triplet_ind:
                if a != p:
                    #print(anchor_x[a].shape,positive_x[p].shape,negative_x[n].shape)
                    triplet_tensor.append(torch.stack([anchor_x[a],positive_x[p],negative_x[n]]))
                    triplet_label.append(torch.stack([anchor_y[a],positive_y[p],negative_y[n]]))
                    #break
        #print(len(triplet_tensor))
        triplet_tensor = torch.stack(triplet_tensor)
        triplet_label = torch.stack(triplet_label)
        
        anchor = triplet_tensor[:,0]
        positive = triplet_tensor[:,1]
        negative = triplet_tensor[:,2]
        
        optimizer.zero_grad()
        #print(out.shape,y.shape)
        classification_loss = c_criterion(out,y)
        triplet_loss = t_criterion(anchor,positive,negative)
        loss = classification_loss + triplet_loss
        b_loss.append(loss.cpu().item())
        loss.backward()
        optimizer.step()
    train_loss.append(np.array(b_loss).mean())
    
    model.eval()
    b_loss=[]
    with torch.no_grad():
        for x,y in val_loader:
            x = Variable(x.float()).to('cuda')
            y = Variable(y.type(torch.LongTensor)).to('cuda')
            
            out = model(x)
            loss = c_criterion(out,y)
            b_loss.append(loss.cpu().item())
    val_loss.append(np.array(b_loss).mean())
    
    
    if epoch % 5 == 0 or epoch == 1:
        model.eval()
        with torch.no_grad():
            tst_embedded = []
            tst_y = []
            for x, y in test_loader:
                x = Variable(x.float()).to('cuda')
                embedded = model.embedding(x.view(-1,784)).detach().cpu()
                tst_embedded.append(embedded)
                tst_y.append(y)
            tst_embedded = torch.cat(tst_embedded)
            tst_y = torch.cat(tst_y)
            #print(tst_embedded.shape,tst_y.shape)
            hle = umap.UMAP(random_state=0,metric='euclidean',n_components=2,n_neighbors=20,min_dist=0).fit_transform(tst_embedded)
            c_lst = [plt.cm.nipy_spectral(a) for a in np.linspace(0.0, 1.0, len(np.unique(tst_y)))]
            plt.figure(figsize=(10,10))
            for i in range(0,len(np.unique(tst_y))):
                #print('Class ',i,'x :',hle[tst_y==i,0])
                #print('Class ',i,'y :',hle[tst_y==i,1])
                plt.scatter(hle[tst_y==i,0],hle[tst_y==i,1],label=i,color=c_lst[i])
            plt.legend(loc='best')
            plt.title('UMAP 2D at Epoch'+str(epoch))
            #plt.savefig(f'../result/model_ownAnomaly_{epoch}epoch.png')
            plt.show()
            plt.close()


In [ ]:
plt.plot(train_loss,label='train loss')
plt.plot(val_loss,label='validation loss')
plt.legend()
plt.show()

In [ ]:
import umap.umap_ as umap
#model.eval()
tst_embedded = []
tst_y = []
with torch.no_grad():
    for x, y in test_loader:
        x = Variable(x.float()).to('cuda')
        embedded = model.embedding(x.view(-1,784)).detach().cpu()
        tst_embedded.append(embedded)
        tst_y.append(y)
            
tst_embedded = torch.cat(tst_embedded)
tst_y = torch.cat(tst_y)
#print(tst_embedded.shape,tst_y.shape)
hle = umap.UMAP(random_state=0,metric='euclidean',n_components=2,n_neighbors=20,min_dist=0).fit_transform(tst_embedded)
c_lst = [plt.cm.nipy_spectral(a) for a in np.linspace(0.0, 1.0, len(np.unique(tst_y)))]
plt.figure(figsize=(10,10))
for i in range(0,len(np.unique(tst_y))):
    #print('Class ',i,'x :',hle[tst_y==i,0])
    #print('Class ',i,'y :',hle[tst_y==i,1])
    plt.scatter(hle[tst_y==i,0],hle[tst_y==i,1],label=i,color=c_lst[i])
plt.legend(loc='best')
plt.title('UMAP 2D at 3 epoch')
plt.show()